In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Lame as hell, but might as well submit it, the submission chance resets anyway

In [6]:
# Load the data
train_df = pd.read_csv("/kaggle/input/MABe-mouse-behavior-detection/train.csv")
print(train_df.size)
print(train_df.shape)
train_tracking_df = pd.read_parquet("/kaggle/input/MABe-mouse-behavior-detection/train_tracking/AdaptableSnail/1212811043.parquet")
train_annotation_df = pd.read_parquet("/kaggle/input/MABe-mouse-behavior-detection/train_annotation/AdaptableSnail/1212811043.parquet")
train_tracking_df.head(20)

333982
(8789, 38)


,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012
5,0,1,nose,527.765015,342.686005
6,0,1,tail_base,477.536011,398.338013
7,0,1,tail_midpoint,446.834015,429.035004
8,0,1,tail_tip,424.618011,457.161011
9,0,2,body_center,702.916016,322.872986


In [49]:
# Preprocessing data
# most important and most painful part of the job

# this code: Annotation label frame for each action are in the form [a to b] in 1 row 
# -> split it into a - b separate rows

# There are action on self and on target? they should be separated and trained separately

frame_labels = []
for _, row in train_annotation_df.iterrows():
    for frame in range(row["start_frame"], row["stop_frame"] + 1):
        frame_labels.append({
            "video_frame": frame,
            "mouse_id" : row["agent_id"],
            "target_id" : int(row["target_id"]),
            "action" : row["action"]
        })
        
frame_labels_df = pd.DataFrame(frame_labels)
frame_labels_df.head()


,video_frame,mouse_id,target_id,action
0,2,1,3,chase
1,3,1,3,chase
2,4,1,3,chase
3,5,1,3,chase
4,6,1,3,chase


In [50]:
# Merge tracking and annotation? IDK if I should
merged_df = train_tracking_df.merge(
    frame_labels_df,
    on=['video_frame', 'mouse_id'],
    how = "left"
)
merged_df['action'] = merged_df['action'].fillna("None").astype(str)
rows = merged_df[merged_df['video_frame'] == 2]
print(rows)


     video_frame  mouse_id       bodypart           x           y  target_id  \
70             2         1    body_center  495.295013  376.574005        3.0   
71             2         1       ear_left  493.692993  344.872986        3.0   
72             2         1      ear_right  517.568970  369.208008        3.0   
73             2         1   lateral_left  474.131989  370.510986        3.0   
74             2         1  lateral_right  505.371002  394.701996        3.0   
75             2         1           nose  528.192993  345.684998        3.0   
76             2         1      tail_base  477.053009  399.122986        3.0   
77             2         1  tail_midpoint  447.006012  429.868011        3.0   
78             2         1       tail_tip  424.605011  457.080994        3.0   
79             2         2    body_center  703.031982  323.678986        NaN   
80             2         2       ear_left  726.744019  302.808990        NaN   
81             2         2      ear_righ

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
# Feature engineering? 
# we want to create more features from existing features.

In [47]:
# Train
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df_numeric = pd.get_dummies(merged_df, columns=['bodypart'])
print(df_numeric.head())
feature_cols = [c for c in df_numeric.columns if c not in ['video_frame', 'mouse_id', 'target_id', 'action']]
X = df_numeric[feature_cols]
y = df_numeric['action']
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

   video_frame  mouse_id           x           y action  bodypart_body_center  \
0            0         1  496.187012  376.475006   None                  True   
1            0         1  494.059998  343.924011   None                 False   
2            0         1  518.765015  367.362000   None                 False   
3            0         1  474.536987  370.563995   None                 False   
4            0         1  505.825012  394.937012   None                 False   

   bodypart_ear_left  bodypart_ear_right  bodypart_lateral_left  \
0              False               False                  False   
1               True               False                  False   
2              False                True                  False   
3              False               False                   True   
4              False               False                  False   

   bodypart_lateral_right  bodypart_neck  bodypart_nose  bodypart_tail_base  \
0                   False      

RandomForestClassifier(n_estimators=10, random_state=42)

In [48]:
# Validate
from sklearn.metrics import classification_report, f1_score
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

              precision    recall  f1-score   support

        None      0.971     0.993     0.982    514952
      attack      0.573     0.260     0.358      8457
       avoid      0.580     0.247     0.346      5923
       chase      0.527     0.196     0.286      4712
 chaseattack      0.639     0.297     0.406      1480

    accuracy                          0.965    535524
   macro avg      0.658     0.399     0.476    535524
weighted avg      0.956     0.965     0.957    535524



ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
# Predict and Format submission (not done)
test_tracking_df = pd.read_parquet("/kaggle/input/MABe-mouse-behavior-detection/test_tracking/AdaptableSnail/438887472.parquet")
df_dummies = pd.get_dummies(test_tracking_df, columns=['bodypart'])
X_test = df_dummies[feature_cols]
y_test = model.predict(X_test)
